In [1]:
import numpy as np
import matplotlib.pyplot as plt

from spatial_pp import SPP_Thomas
from whittle_estimator import ThomasWhittleEstimator

In [2]:
def standardise(spp, W_x, W_y):
    """
    Function to standardise the coordinates of a given spp.
    """
    spp[:,0] = spp[:,0]/W_x; spp[:,1] = spp[:,1]/W_y

    return spp

In [3]:
# Parameter values for point process
rho = 25
K = 10
sigma = 0.03
cov = np.array([[1, 0], [0, 1]])

# Number of repetitions per side length
N_w = int(1e2) 

# Iterate over a set of side lengths
side_lengths = np.arange(1, 5 + 1, 0.5)

# Parameter array: 0,1,2 for rho, K, sigma; 3 for number of points
# in the pattern and 4 for the edge length of the square.
params = np.zeros((N_w * len(side_lengths), 5))

for l in side_lengths:

    # Print value of l for tracking
    print(f"Side length: {l}")

    # Instatiate class for Thomas process on the given window
    tom = SPP_Thomas(maxX=l, maxY=l)  

    for k in range(N_w):

        # Print iteration
        if (((k+1) / 25) == ((k+1) // 25)):
            print(f"Run {k + 1} of {N_w}")

        # Simulate a point pattern and standardise the coorindates.
        spp = tom.simSPP(rho, K, sigma, cov, enlarge=1.1)
        spp = standardise(spp, l, l)
        
        # Store side length and number of points
        params[k, 3] = len(spp)
        params[k, 4] = l

        # Implement Whittle estimator
        twe = ThomasWhittleEstimator(spp, -16, 16, 1)
        params[k, 0:3] = (twe.scipyOptimisation([15, 5, 0.05]))

Side length: 1.0
Run 25 of 100
Run 50 of 100
Run 75 of 100
Run 100 of 100
Side length: 1.5
Run 25 of 100
Run 50 of 100
Run 75 of 100
Run 100 of 100
Side length: 2.0
Run 25 of 100
Run 50 of 100
Run 75 of 100
Run 100 of 100
Side length: 2.5
Run 25 of 100
Run 50 of 100
Run 75 of 100
Run 100 of 100
Side length: 3.0
Run 25 of 100
Run 50 of 100
Run 75 of 100
Run 100 of 100
Side length: 3.5


KeyboardInterrupt: 

In [4]:
import pandas as pd

df_side_length = pd.DataFrame({
    'rho': (params[:,0] - rho)/rho,
    'K': (params[:,1] - K)/K,
    'sigma': (params[:,2] - sigma)/sigma,
    'N_samp': params[:,3],
    'side_length': params[:,4]
})

side_length_se = df_side_length.groupby('side_length').mean()
